<a href="https://colab.research.google.com/github/ULDBerry/Budget/blob/master/ET5003_KaggleCompetition_DBerry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<img src="https://drive.google.com/uc?export=view&id=1vK33e_EqaHgBHcbRV_m38hx6IkG0blK_" width="350"/>
</div> 

#**Artificial Intelligence - MSc**
##ET5003 - MACHINE LEARNING APPLICATIONS 

###Instructor: Enrique Naredo
###ET5003_KaggleCompetition

In [ ]:
#@title Current Date
Today = '2021-10-05' #@param {type:"date"}


In [ ]:
#@markdown ---
#@markdown ### Enter your details here:
Team_Number = "6" #@param {type:"string"}
Student_ID&Name = "" #@param {type:"string"}
Student_ID&Name = "" #@param {type:"string"}
Student_ID&Name = "" #@param {type:"string"}
Student_ID&Name = "" #@param {type:"string"}
Student_ID&Name = "" #@param {type:"string"}
#@markdown ---

In [ ]:
#@title Notebook information
Notebook_type = 'Etivity' #@param ["Example", "Lab", "Practice", "Etivity", "Assignment", "Exam"]
Version = Final #@param ["Draft", "Final"] {type:"raw"}
Submission = False #@param {type:"boolean"}

# INTRODUCTION

Your introduction here.

***The goal is to use advanced Machine Learning methods to predict House price.***

## Imports

In [1]:
# Suppressing Warnings:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# to plot
import matplotlib.colors
from mpl_toolkits.mplot3d import Axes3D

# to generate classification, regression and clustering datasets
import sklearn.datasets as dt

# to create data frames
from pandas import DataFrame

# to generate data from an existing dataset
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV

In [4]:
# Scikit-learn is an open source machine learning library 
# that supports supervised and unsupervised learning
# https://scikit-learn.org/stable/
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix

In [5]:
# Regular expression operations
#https://docs.python.org/3/library/re.html
import re 

# Natural Language Toolkit
# https://www.nltk.org/install.html
import nltk

# Stemming maps different forms of the same word to a common “stem” 
# https://pypi.org/project/snowballstemmer/
from nltk.stem import SnowballStemmer

# https://www.nltk.org/book/ch02.html
from nltk.corpus import stopwords

# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
from sklearn.preprocessing import StandardScaler

In [6]:
!pip install gpy

     |████████████████████████████████| 959 kB 7.5 MB/s 
     |████████████████████████████████| 71 kB 8.0 MB/s 
  Created wheel for gpy: filename=GPy-1.10.0-cp37-cp37m-linux_x86_64.whl size=2565095 sha256=24a0a2ebdbfaa0673a7fc65c056f03e1ea1bd6242bc7faee10897ca9279f65fb
  Stored in directory: /root/.cache/pip/wheels/f7/18/28/dd1ce0192a81b71a3b086fd952511d088b21e8359ea496860a
  Created wheel for paramz: filename=paramz-0.9.5-py3-none-any.whl size=102565 sha256=5ef34aa1b49f0c5cfeb1508e8b75a1f91565b3c66a1b4d8dc6af01bf57d2339a
  Stored in directory: /root/.cache/pip/wheels/c8/95/f5/ce28482da28162e6028c4b3a32c41d147395825b3cd62bc810
Successfully built gpy paramz


In [7]:
import GPy as GPy
import numpy as np
import pylab as pb
import pymc3 as pm
import arviz as az

In [ ]:
# Define the seed so that results can be reproduced
seed = 11
rand_state = 11

# Define the color maps for plots
color_map = plt.cm.get_cmap('RdYlBu')
color_map_discrete = matplotlib.colors.LinearSegmentedColormap.from_list("", ["red","cyan","magenta","blue"])

# DATASET

Extract from this [paper](https://ieeexplore.ieee.org/document/9300074):

* House prices are a significant impression of the economy, and its value ranges are of great concerns for the clients and property dealers. 

* Housing price escalate every year that eventually reinforced the need of strategy or technique that could predict house prices in future. 

* There are certain factors that influence house prices including physical conditions, locations, number of bedrooms and others.


1. [Download the dataset](https://github.com/UL-CS6134/CS6134_SEM1_2021-2/tree/main/Week-5). 

2. Upload the dataset into your folder.



The challenge is to predict the final price of each house.

## Load the data from google drive

In [8]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# path to your (local/cloud) drive 
path = '/content/drive/MyDrive/Colab Notebooks/Data/Kaggle/'

# load dataset
df_train = pd.read_csv(path+'HousePrice_Train.csv', encoding='utf-8')
df_test = pd.read_csv(path+'HousePrice_Test.csv', encoding='utf-8')



## Exploratory Data Analysis

In [10]:
# get a list of features (column) names 
df_train.columns

Index(['Index', 'ID', 'Location', 'Num_Bathrooms', 'Num_Beds', 'BER_class',
       'Description', 'Services', 'Features', 'Latitude', 'Longitude', 'Type',
       'Surface', 'Price'],
      dtype='object')

In [11]:
# show first 10 rows 
df_train.head(10)

,Index,ID,Location,Num_Bathrooms,Num_Beds,BER_class,Description,Services,Features,Latitude,Longitude,Type,Surface,Price
0,1,12409116,Ongar,2,2,C3,DNG are delighted to present to the market no....,NaN,None,53.396659,-6.438791,apartment,67.00,195000
1,2,12320330,North Strand,1,2,D1,13 Northbrook Terrace is a charming red brick ...,NaN,Bight and Spacious interior\nNumerous original...,53.357129,-6.239779,terraced,95.97,425000
2,3,12405953,Stepaside,3,3,A3,"Welcome to No. 6 Cotter Close, an immaculately...",NaN,Special Features\nLarge rear garden\nGenerous ...,53.266468,-6.222771,semi-detached,107.00,535000
3,4,12202582,Cabinteely,1,3,E1,Welcome to 3 Sycamore Walk!\n\r\nEnjoying a pr...,NaN,Wonderful 3 bed detached bungalow. \nAccom...,53.261600,-6.166738,detached,81.00,499000
4,5,12299336,Lusk,3,4,D1,'Four Seasons' comes to the market as a wonder...,NaN,Detached Property \nDouble Glazed Windows...,53.550731,-6.121415,detached,153.00,510000
5,6,12225979,Glasnevin,1,3,E1,Property Team Lappin Estates take great pleasu...,NaN,None,53.380367,-6.279310,terraced,140.00,360000
6,7,12209989,North Circular Road,1,2,C3,6 Marlborough Court is a bright and spacious t...,NaN,First floor purpose built apartment\nCul de sa...,53.355659,-6.294885,apartment,70.98,300000
7,8,12084555,Clondalkin,2,4,NaN,An outstanding detached Victorian Villa dating...,NaN,Magnificent detached Victorian Villa dating fr...,53.318857,-6.391826,detached,244.98,590000
8,9,12405966,Finglas,3,2,NaN,Ray Cooke Auctioneers are excited to present t...,NaN,None,53.400071,-6.304156,terraced,72.00,229000
9,10,12202195,Howth,2,2,NaN,Needles Cottage is located on the southern slo...,NaN,None,53.366407,-6.077866,semi-detached,63.70,499500


In [17]:
# What is meant by 'Services ?'
df_train['Services'].unique()

# Parking & Alarm interesting
# Television services and type of heating not so much


array([nan, 'Parking,Gas Fired Central Heating,Alarm',
       'Parking,Wired for Cable Television',
       'Parking,Gas Fired Central Heating,Alarm,Wired for Cable Television,Wheelchair Access',
       'Parking,Gas Fired Central Heating,Wired for Cable Television',
       'Parking,Oil Fired Central Heating',
       'Parking,Gas Fired Central Heating,Alarm,Wired for Cable Television',
       'Gas Fired Central Heating,Wired for Cable Television', 'Parking',
       'Gas Fired Central Heating', 'Parking,Gas Fired Central Heating',
       'Parking,Gas Fired Central Heating,Wired for Cable Television,Wheelchair Access',
       'Parking,Gas Fired Central Heating,Wheelchair Access',
       'Parking,Alarm,Wired for Cable Television,Oil Fired Central Heating',
       'Parking,Alarm,Wired for Cable Television',
       'Wired for Cable Television',
       'Parking,Alarm,Oil Fired Central Heating',
       'Parking,Gas Fired Central Heating,Alarm,Wired for Cable Television,Oil Fired Central Heating

In [18]:
# Lets check for NaN's
df_train.isna().sum()

Index               0
ID                  0
Location            0
Num_Bathrooms       0
Num_Beds            0
BER_class         251
Description         0
Services         1061
Features            0
Latitude            0
Longitude           0
Type                0
Surface             0
Price               0
dtype: int64

In [28]:
 # Check to NaN's against row count to see if we can do something to generate data in the NaNs
 # BER is an important feature for price prediction 

round((df_train.isna().sum()/df_train.shape[0])*100,0)

Index             0.0
ID                0.0
Location          0.0
Num_Bathrooms     0.0
Num_Beds          0.0
BER_class        15.0
Description       0.0
Services         65.0
Features          0.0
Latitude          0.0
Longitude         0.0
Type              0.0
Surface           0.0
Price             0.0
dtype: float64

As just 15 % of BER_Class values are missing I will attempt to create those values
While Services could prove a useful feature for NLP analysis there are to many (65%) missing values .  I will drop this feature 


In [33]:
df_train['Location'].unique()

array(['Ongar', 'North Strand', 'Stepaside', 'Cabinteely', 'Lusk',
       'Glasnevin', 'North Circular Road', 'Clondalkin', 'Finglas',
       'Howth', 'Rathfarnham', 'Clontarf', 'Balbriggan', 'Killester',
       'Rathmines', 'Baldoyle', 'Rush', 'Dublin 2', 'Artane',
       'Ballycullen', 'Loughlinstown', 'Ballsbridge', 'Lucan',
       'Balgriffin', 'Navan Road (D7)', 'Castleknock', 'Dun Laoghaire',
       'Whitehall', 'Rathcoole', 'Dublin 1', 'Templeogue', 'Beaumont',
       'Sandycove', 'Kinsealy', 'Perrystown', 'Drumcondra',
       'Blanchardstown', 'Kilmainham', 'Monkstown', 'Rathgar', 'Sutton',
       'Citywest', 'Milltown', 'Malahide', 'Ringsend', 'Blackrock',
       'Clonsilla', 'Sandyford', 'Kilmacud', 'Glenageary', 'Clongriffin',
       'Skerries', 'Ranelagh', 'IFSC', 'Terenure', 'Dundrum',
       'Sandymount', 'Donaghmede', 'Dublin 7', 'Knocklyon', 'Killiney',
       'Santry', 'Tallaght', 'Palmerstown', 'Cabra', 'Ballinteer',
       'Donabate', 'Kilnamanagh', 'Rathmichael', 'S

In [14]:

df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
Index,1638.0,8.195000e+02,472.994186,1.000000e+00,4.102500e+02,8.195000e+02,1.228750e+03,1.638000e+03
ID,1638.0,1.231930e+07,144754.041968,1.118567e+07,1.228104e+07,1.238055e+07,1.240548e+07,1.242836e+07
Num_Bathrooms,1638.0,2.054335e+00,1.241259,1.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,1.800000e+01
Num_Beds,1638.0,3.043346e+00,1.379650,1.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00,2.700000e+01
Latitude,1638.0,5.335047e+01,0.076832,5.250186e+01,5.329426e+01,5.334139e+01,5.338203e+01,5.363059e+01
Longitude,1638.0,-6.255086e+00,0.144430,-6.497000e+00,-6.309613e+00,-6.247780e+00,-6.193982e+00,-1.744995e+00
Surface,1638.0,1.695407e+02,1791.793934,3.400000e+00,7.300000e+01,9.705000e+01,1.350000e+02,7.223639e+04
Price,1638.0,5.547196e+05,565203.162814,1.999500e+04,2.950000e+05,3.950000e+05,5.950000e+05,8.900000e+06


In [15]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1638 entries, 0 to 1637
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Index          1638 non-null   int64  
 1   ID             1638 non-null   int64  
 2   Location       1638 non-null   object 
 3   Num_Bathrooms  1638 non-null   int64  
 4   Num_Beds       1638 non-null   int64  
 5   BER_class      1387 non-null   object 
 6   Description    1638 non-null   object 
 7   Services       577 non-null    object 
 8   Features       1638 non-null   object 
 9   Latitude       1638 non-null   float64
 10  Longitude      1638 non-null   float64
 11  Type           1638 non-null   object 
 12  Surface        1638 non-null   float64
 13  Price          1638 non-null   int64  
dtypes: float64(3), int64(5), object(6)
memory usage: 179.3+ KB


In [ ]:
# Lets look for NaN's




## Training & Test Data

In [ ]:
# split data into training and test
from sklearn.model_selection import train_test_split

# training: 70% (0.7), test: 30% (0.3) 
# you could try any other combination 
# but consider 50% of training as the low boundary
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3)

### Train dataset

In [ ]:
# show first data frame rows 
dftrain.head()

In [ ]:
# Generate descriptive statistics
dftrain.describe()

### Test dataset

In [ ]:
# show first data frame rows 
dftest.head()

In [ ]:
# Generate descriptive statistics
dftest.describe()

### Expected Cost dataset

# NATURAL LANGUAGE PROCESSING

Natural language processing ([NLP](https://en.wikipedia.org/wiki/Natural_language_processing)) is a subfield of linguistics, computer science, and artificial intelligence.

* NLP concerned with the interactions between computers and human language.
* In particular how to program computers to process and analyze large amounts of natural language data. 
* The goal is a computer capable of "understanding" the contents of documents.
* Including the contextual nuances of the language within them. 
* The technology can then accurately extract information and insights contained in the documents.
* As well as categorize and organize the documents themselves. 

Your comments, explanation, and references here.

## Pre processing training and testing datasets

In [ ]:
# Using English Language
# Removing stopwords and stemming
# a stem must be a word
# Example:  fishing, fished, and fisher: stem -> fish
# Stop words are basically a set of commonly used words in any language
# and are filtered out before processing of natural language data 
stemmer = SnowballStemmer('english', ignore_stopwords=False)
nltk.download('stopwords')
stop = set(stopwords.words('english'))


# PIECEWISE REGRESSION

**Piecewise regression**, extract from [Wikipedia](https://en.wikipedia.org/wiki/Segmented_regression):

Segmented regression, also known as piecewise regression or broken-stick regression, is a method in regression analysis in which the independent variable is partitioned into intervals and a separate line segment is fit to each interval. 

* Segmented regression analysis can also be performed on 
multivariate data by partitioning the various independent variables. 
* Segmented regression is useful when the independent variables, clustered into different groups, exhibit different relationships between the variables in these regions. 

* The boundaries between the segments are breakpoints.

* Segmented linear regression is segmented regression whereby the relations in the intervals are obtained by linear regression. 

Your comments, explanation, and references here.

In [ ]:
# your code here

# BAYESIAN NN

A [Bayesian network](https://en.wikipedia.org/wiki/Bayesian_network) (also known as a Bayes network, Bayes net, belief network, or decision network) is a probabilistic graphical model that represents a set of variables and their conditional dependencies via a directed acyclic graph (DAG). 

* Bayesian networks are ideal for taking an event that occurred and predicting the likelihood that any one of several possible known causes was the contributing factor. 
* For example, a Bayesian network could represent the probabilistic relationships between diseases and symptoms. 
* Given symptoms, the network can be used to compute the probabilities of the presence of various diseases.

Your comments, explanation, and references here.

In [ ]:
# your code here

## Output the prediction 


In [ ]:
# ModelName.to_csv("FileName")
Decision.to_csv("Post_Kaggle_Decison.csv",  index=False)

## References

https://thispointer.com/pandas-get-unique-values-in-single-or-multiple-columns-of-a-dataframe-in-python/

https://stackoverflow.com/questions/15943769/how-do-i-get-the-row-count-of-a-pandas-dataframe


# SUMMARY

Your summary here.